In [ ]:
import pandas as pd
import sqlite3
import psycopg

In [ ]:
providers_df = pd.read_csv('providers_data.csv')
receivers_df = pd.read_csv('receivers_data.csv')
food_listings_df = pd.read_csv('food_listings_data.csv')
claims_df = pd.read_csv('claims_data.csv')

In [ ]:
providers_df

In [ ]:
receivers_df

In [ ]:
food_listings_df

In [ ]:
claims_df

In [ ]:
print(food_listings_df.info())
print(food_listings_df.head(5))

In [ ]:
!pip install psycopg
import psycopg
!pip install psycopg2
import psycopg2

In [ ]:
DB_CONFIG = {
    "dbname" == "ramji",
    "username" == "postgres",
    "password" == "Ktm@9291",
    "host" == "localhost",
    "port" == 5432
}

connection = psycopg2.connect(host = "localhost",database = "ramji", user = "postgres" , password ="Ktm@9291")
cur = connection.cursor()


In [ ]:
# Create providers table
cur.execute("""
CREATE TABLE IF NOT EXISTS providers (
Provider_ID INTEGER PRIMARY KEY,
Name TEXT,
Type TEXT,
Address TEXT,
City TEXT,
Contact TEXT
);
""")
connection.commit()
connection.autocommit = True

In [ ]:
# Create receivers table
cur.execute("""
CREATE TABLE IF NOT EXISTS receivers (
Receiver_ID INTEGER PRIMARY KEY,
Name TEXT,
Type TEXT,
City TEXT,
Contact TEXT
);
""")

In [ ]:
# Create food_listings table with foreign key linking to providers
cur.execute("""
CREATE TABLE IF NOT EXISTS food_listings (
Food_ID INTEGER PRIMARY KEY,
Food_Name TEXT,
Quantity INTEGER,
Expiry_Date TEXT,
Provider_ID INTEGER,
Provider_Type TEXT,
Location TEXT,
Food_Type TEXT,
Meal_Type TEXT,
FOREIGN KEY(Provider_ID) REFERENCES providers(Provider_ID));
""")

In [ ]:
# Create claims table with foreign keys to food_listings and receivers
cur.execute("""
CREATE TABLE IF NOT EXISTS claims (
Claim_ID INTEGER PRIMARY KEY,
Food_ID INTEGER,
Receiver_ID INTEGER,
Status TEXT,
Timestamp TEXT,
FOREIGN KEY(Food_ID) REFERENCES food_listings(Food_ID),
FOREIGN KEY(Receiver_ID) REFERENCES receivers(Receiver_ID)
);
""")

In [ ]:
# 1. How many food providers and receivers are there in each city?
query = "select City, count(*) from providers group by City"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)
# query = "select City, count(*) 'Nos of Receivers' from receivers group by City"
# receivers_count = pd.read_sql_query(query, connection)
# print(receivers_count.head())
query = "select City, count(*) from receivers group by City"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 2. Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query = "select Provider_Type, SUM(Quantity) from food_listings, providers where food_listings.Provider_ID = providers.Provider_ID group by Provider_Type order by SUM(Quantity) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# # # 3. What is the contact information of food providers in a specific city?
# query = "select  city, count(*) from providers  group by City SELECT Name,city,Contact FROM providers WHERE city IN (SELECT city FROM providers GROUP BY city HAVING COUNT(*) > 1) union all SELECT Name, City,Contact FROM providers WHERE city IN (SELECT city FROM providers GROUP BY city HAVING COUNT(*) = 1)"
# cur.execute(query)
# rows = cur.fetchall()
# for row in rows:
#  print (row)


In [ ]:
# 4. Which receivers have claimed the most food?
query = "select  c.Receiver_ID,type,Status,count(*) from receivers r,claims c where r.Receiver_ID= c.Receiver_ID and Status = 'Completed' group by c.Receiver_ID,type,Status having count(*) > 1"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 5. What is the total quantity of food available from all providers?
query = "select SUM(Quantity) from food_listings left join claims on claims.Food_ID = food_listings.Food_ID where Claim_ID is null"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 6. Which city has the highest number of food listings?

query = "select fl.provider_ID,city, SUM(Quantity) from providers P, food_listings FL where p.Provider_ID = FL.Provider_ID group by fl.Provider_ID,City order by SUM(Quantity) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 7. What are the most commonly available food types?

query = "SELECT Food_Type, COUNT(*) AS Availability_Count FROM Food_Listings GROUP BY Food_Type ORDER BY Availability_Count DESC"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 8. How many food claims have been made for each food item?

query = "select f.food_name,count(Claim_ID) as claims  from providers p, food_listings f,claims c where p.Provider_ID = f.Provider_ID and f.Food_ID = c.Food_ID group by f.Food_Name order by claims  desc "
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 9. Which provider has had the highest number of successful food claims?

query = "select p.Provider_ID,count(*),Name from providers p, food_listings f,claims c where p.Provider_ID = f.Provider_ID and f.Food_ID = c.Food_ID group by p.Provider_ID,name having count(*) > 1 order by count(*) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
# 10. What percentage of food claims are completed vs. pending vs. canceled?

query = "SELECT status,COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims) FROM claims GROUP BY status;"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)
# 11. What is the average quantity of food claimed per receiver?

query = "SELECT r.Name, AVG(quantity) AS Avg FROM claims c,food_listings f, receivers r where c.Food_ID =f.Food_ID and c.Receiver_ID = r.Receiver_ID GROUP BY r.Name order by Avg desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


In [ ]:
# 11. What is the average quantity of food claimed per receiver?

query = "SELECT r.Name, AVG(quantity) AS Avg FROM claims c,food_listings f, receivers r where c.Food_ID =f.Food_ID and c.Receiver_ID = r.Receiver_ID GROUP BY r.Name order by Avg desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


In [ ]:
# 12. Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

query = "select Meal_Type, count(*) from  food_listings f,claims c where f.Food_ID = c.Food_ID group by Meal_Type having count(*) > 1 order by count(*) desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)


In [ ]:
# 13. What is the total quantity of food donated by each provider?

query = "SELECT p.Name,sum(quantity) as total from providers p, food_listings f where p.Provider_ID = f.Provider_ID group by p.Name order by total desc"
cur.execute(query)
rows = cur.fetchall()
for row in rows:
    print (row)

In [ ]:
import streamlit as st
!pip install streamlit

In [ ]:
%%writefile Food_Waste_Project_finalDraft_app.py
import streamlit as st

# Set Streamlit page config
st.set_page_config(page_title="Local Food Wastage System", layout="wide")

# Page title
st.title("🍛 Local Food Wastage Management System")

# Sidebar navigation
menu = st.sidebar.radio("Select Section", ["Providers", "Receivers", "Food Listings", "Claims", "Analysis"])


In [ ]:
!streamlit run Food_Waste_Project_finalDraft_app.py

In [ ]:

# Providers Section
if menu == "Providers":
    st.header("Food Providers by City")
    cities = pd.read_sql_query("SELECT DISTINCT City FROM providers_data ORDER BY City", conn)
    selected_city = st.selectbox("Select a City", cities['City'])

    query = "SELECT Name, Contact FROM providers_data WHERE City = "
    results = pd.read_sql_query(query, conn, params=(selected_city,))
    st.write(f"Found {len(results)} providers in {selected_city}:")
    st.dataframe(results)
    

In [ ]:
# Receivers Section
elif menu == "Receivers":
    st.header("🎯 Food Receivers by City")
    cities = pd.read_sql_query("SELECT DISTINCT City FROM receivers_data ORDER BY City", conn)
    selected_city = st.selectbox("Select a City", cities['City'])

    query = "SELECT Name, Contact FROM receivers_df WHERE City = ?"
    results = pd.read_sql_query(query, conn, params=(selected_city,))
    st.write(f"Found {len(results)} receivers in {selected_city}:")
    st.dataframe(results)


In [ ]:
# Food Listings Section
elif menu == "Food Listings":
    st.header("🍽 Available Food Listings")
    df = pd.read_sql_query("SELECT * FROM food_listings_data", conn)
    st.dataframe(df)


In [ ]:
# Claims Section
elif menu == "Claims":
    st.header("📦 Food Claims Overview")
    df = pd.read_sql_query("SELECT * FROM claims_data", conn)
    st.dataframe(df)


In [ ]:
# Analysis Section
elif menu == "Analysis":
    st.header("📊 Insights & SQL Queries")

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("1. Providers & Receivers by City")
        df1 = pd.read_sql_query("""
            SELECT City, COUNT(*) AS Providers FROM providers_data GROUP BY City""", conn)
        df2 = pd.read_sql_query("""
            SELECT City, COUNT(*) AS Receivers FROM receivers_data GROUP BY City """, conn)
        merged = pd.merge(df1, df2, on="City", how="outer").fillna(0)
        st.dataframe(merged)

    with col2:
        st.subheader("2. Top Provider Types")
        df = pd.read_sql_query("""SELECT Type, COUNT(*) AS Total FROM providers_data GROUP BY Type ORDER BY Total DESC""", conn)
        st.dataframe(df)

    st.subheader("3. Completed vs. Pending vs. Cancelled Claims")
    df = pd.read_sql_query(""" SELECT Status, COUNT(*) AS Count FROM claims_data GROUP BY Status """, conn)
    st.bar_chart(df.set_index('Status'))

conn.close()

In [ ]:
!streamlit run Food_Waste_Project_finalDraft_app.py